In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
cube = Lar.cuboid([1,1,1])

([0.0 0.0 … 1.0 1.0; 0.0 0.0 … 1.0 1.0; 0.0 1.0 … 0.0 1.0], [[1, 2, 3, 4, 5, 6, 7, 8]])

## Vecchia funzione

In [5]:
function apply(affineMatrix, larmodel)
	data = collect(larmodel)
	V = data[1]

	m,n = size(V)
	W = [V; fill(1.0, (1,n))]
	V = (affineMatrix * W)[1:m,1:n]

	data[1] = V
	larmodel = Tuple(data)
	return larmodel
end

apply (generic function with 1 method)

In [6]:
@btime apply(Lar.t(-0.5,-0.5),square)

  1.620 μs (14 allocations: 1.16 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [7]:
@btime apply(Lar.t(-0.5,-0.5,-0.5),cube)

  1.710 μs (14 allocations: 1.67 KiB)


([-0.5 -0.5 … 0.5 0.5; -0.5 -0.5 … 0.5 0.5; -0.5 0.5 … -0.5 0.5], [[1, 2, 3, 4, 5, 6, 7, 8]])

In [8]:
@benchmark apply(Lar.t(-0.5,-0.5,-0.5),cube)

BenchmarkTools.Trial: 
  memory estimate:  1.67 KiB
  allocs estimate:  14
  --------------
  minimum time:     1.710 μs (0.00% GC)
  median time:      1.810 μs (0.00% GC)
  mean time:        2.495 μs (4.15% GC)
  maximum time:     414.630 μs (97.98% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [9]:
@code_warntype apply(Lar.t(-0.5,-0.5,-0.5),cube) 

Variables
  #self#::Core.Compiler.Const(apply, false)
  affineMatrix::Array{Float64,2}
  larmodel@_3::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}
  @_4::Any
  data::Array{Array,1}
  m::Any
  n::Any
  W::Any
  V::Any
  larmodel@_10::Tuple{Vararg{Array,N} where N}

Body::Tuple{Vararg{Array,N} where N}
1 ─       (larmodel@_10 = larmodel@_3)
│         (data = Main.collect(larmodel@_10::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}))
│         (V = Base.getindex(data, 1))
│   %4  = Main.size(V::Array)::Any
│   %5  = Base.indexed_iterate(%4, 1)::Any
│         (m = Core.getfield(%5, 1))
│         (@_4 = Core.getfield(%5, 2))
│   %8  = Base.indexed_iterate(%4, 2, @_4)::Any
│         (n = Core.getfield(%8, 1))
│   %10 = V::Array::Array
│   %11 = Core.tuple(1, n)::Core.Compiler.PartialStruct(Tuple{Int64,Any}, Any[Core.Compiler.Const(1, false), Any])
│   %12 = Main.fill(1.0, %11)::Array{Float64,2}
│         (W = Base.vcat(%10, %12))
│   %14 = (affineMatrix * W)::Any
│   %15 = (1:m)::Any
│  

## Modifiche

In [4]:
function applyv(affineMatrix, larmodel)
	data = collect(larmodel)
	V = data[1]

	m,n = size(V)
	W = [V; fill(1.0, (1,n))]
	V = (affineMatrix * W)[1:m,1:n]

	data[1] = V
	larmodel = Tuple(data)
	return larmodel
end

applyv (generic function with 1 method)

In [21]:
@btime applyv(Lar.t(-0.5,-0.5,-0.5),cube)

  1.720 μs (14 allocations: 1.67 KiB)


([-0.5 -0.5 … 0.5 0.5; -0.5 -0.5 … 0.5 0.5; -0.5 0.5 … -0.5 0.5], [[1, 2, 3, 4, 5, 6, 7, 8]])

In [12]:
@code_typed applyv(Lar.t(-0.5,-0.5,-0.5),cube)

CodeInfo(
1 ─ %1  = π (larmodel@_3, Tuple{Array{Float64,2},Array{Array{Int64,1},1}})
│   %2  = invoke Base._collect($(QuoteNode(1:1))::UnitRange{Int64}, %1::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}, $(QuoteNode(Base.HasEltype()))::Base.HasEltype, $(QuoteNode(Base.HasLength()))::Base.HasLength)::Array{Array,1}
│   %3  = Base.arrayref(true, %2, 1)::Array
│   %4  = Main.size(%3)::Any
│   %5  = Base.indexed_iterate(%4, 1)::Any
│   %6  = Core.getfield(%5, 1)::Any
│   %7  = Core.getfield(%5, 2)::Any
│   %8  = Base.indexed_iterate(%4, 2, %7)::Any
│   %9  = Core.getfield(%8, 1)::Any
│   %10 = Core.tuple(1, %9)::Core.Compiler.PartialStruct(Tuple{Int64,Any}, Any[Core.Compiler.Const(1, false), Any])
│   %11 = Main.fill(1.0, %10)::Array{Float64,2}
│   %12 = Base.vcat(%3, %11)::Any
│   %13 = (affineMatrix * %12)::Any
│   %14 = (isa)(%6, Int64)::Bool
└──       goto #3 if not %14
2 ─ %16 = π (%6, Int64)
│   %17 = Base.sle_int(1, %16)::Bool
│   %18 = Base.sub_int(1, 1)::Int64
│   %19 = Base.ife

In [13]:
@code_warntype applyv(Lar.t(-0.5,-0.5,-0.5),cube)

Variables
  #self#::Core.Compiler.Const(applyv, false)
  affineMatrix::Array{Float64,2}
  larmodel@_3::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}
  @_4::Any
  data::Array{Array,1}
  m::Any
  n::Any
  W::Any
  V::Any
  larmodel@_10::Tuple{Vararg{Array,N} where N}

Body::Tuple{Vararg{Array,N} where N}
1 ─       (larmodel@_10 = larmodel@_3)
│         (data = Main.collect(larmodel@_10::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}))
│         (V = Base.getindex(data, 1))
│   %4  = Main.size(V::Array)::Any
│   %5  = Base.indexed_iterate(%4, 1)::Any
│         (m = Core.getfield(%5, 1))
│         (@_4 = Core.getfield(%5, 2))
│   %8  = Base.indexed_iterate(%4, 2, @_4)::Any
│         (n = Core.getfield(%8, 1))
│   %10 = V::Array::Array
│   %11 = Core.tuple(1, n)::Core.Compiler.PartialStruct(Tuple{Int64,Any}, Any[Core.Compiler.Const(1, false), Any])
│   %12 = Main.fill(1.0, %11)::Array{Float64,2}
│         (W = Base.vcat(%10, %12))
│   %14 = (affineMatrix * W)::Any
│   %15 = (1:m)::Any
│ 